This implimentation is using hourly updates. As SerpAPI only can give us hourly data for the last 7 days, we have to update the data at most 5-6 after the last update for the functions to work.

# Dependencies

In [ ]:
pip install serpapi

In [ ]:
pip install Historic-Crypto

In [ ]:
from Historic_Crypto import LiveCryptoData
from Historic_Crypto import HistoricalData
import pandas as pd

# Uploading data

In [ ]:
# Uploading last data that we have
BTC = pd.read_csv('Bitcoin_update.csv')
LTC = pd.read_csv('Litecoin_update.csv')
ETH = pd.read_csv('Ethereum_update.csv')

# Functions to update data

In [ ]:
import os
import serpapi
def search(key_words):
  '''
  This function searches using SerpAPI for Google Trends, the percentage of how trending are the words from key_words parameter and saves the results
  in a dataframe.
  '''
  client = serpapi.Client(api_key='c2c12deb86e94040bc9aa365762be1ae1d37136b545b20fc2c837b76a673819b')
  search =  client.search(
      engine="google_trends",
      q=key_words,
      api_key='c2c12deb86e94040bc9aa365762be1ae1d37136b545b20fc2c837b76a673819b',
      date = "now 7-d",
    )
  timeline_data_months = search["interest_over_time"]["timeline_data"]
  date_data = {}

  for item in timeline_data_months:
      date = item['date']
      timestamp = item['timestamp']
      values = item['values']
      query_data = {}
      for value_dict in values:
          query = value_dict['query']
          extracted_value = value_dict['extracted_value']
          query_data[query] = extracted_value
      date_data[date] = {'timestamp': timestamp, **query_data}

  df_days = pd.DataFrame(date_data).T.reset_index()
  df_days.rename(columns={'index': 'date'}, inplace=True)
  df_days = df_days[['date', 'timestamp'] + sorted(df_days.columns.difference(['date', 'timestamp']))]
  df_days['time'] = pd.to_datetime(df_days['date'])
  return df_days

In [ ]:
key_words_btc = "crypto, buy bitcoin, cryptocurrency, Blockchain, Bitcoin"
key_words_ltc = "crypto, buy litecoin, cryptocurrency, Blockchain, litecoin"
key_words_eth = "crypto, buy ethereum, cryptocurrency, Blockchain, ethereum"

In [ ]:
def Update_df(data, type_str, key_words, type_name):
  # Using Google Trends we are searching for the given key words and saving it in a df
  df_days = search(key_words)
  data['time'] = pd.to_datetime(data['time'])
  # finding the last date for which we have data
  parameter_date = data.iloc[-1][0]
  # adding one hour to start saving data from that point until now
  time = parameter_date + pd.Timedelta(hours=1)
  data_update = HistoricalData(type_str,3600,time.strftime('%Y-%m-%d-%H-%M'), verbose =  False).retrieve_data()
  data_update.reset_index(inplace=True)
  df1 = data_update.copy()
  df2 = df_days.copy()
  # adding values from search data of kew words to the dataframe with prices
  data_new = pd.merge(df1, df2, left_on='time', right_on='time', how='left')
  # Because Google Trends gives us up to a certaib point of the current day (usualy 10 hours delay)
  # we will find the row starting where we don't have data to add to the prices df
  idx = data_new.index[pd.isna(data_new[type_name])][0]
  # we will get only the part of the dataframe where we have values
  df_before = data_new.iloc[:idx]
  # adding the new optained data to the one we already have
  result_df = pd.concat([data, df_before], ignore_index=True)
  # removing last three columns
  result_df = result_df.iloc[:, :-3]
  # transforming time column in unix timestamp values
  result_df['unix_timestamp'] = result_df['time'].view('int64') // 10**9
  return result_df

In [ ]:
# Making the updates
df1 = Update_df(BTC, "BTC-USD", key_words_btc, 'Bitcoin')
df2 = Update_df(LTC, "LTC-USD", key_words_ltc, 'litecoin')
df3 = Update_df(ETH, "ETH-USD", key_words_eth, 'ethereum')

# Function to get the price

In [ ]:
def get_price(type_name):
  if type_name == 'Bitcoin' or type_name == "bitcoin":
    data = df1
  elif type_name == "Litecoin" or type_name == "litecoin":
    data = df2
  elif type_name == "Ethereum" or type_name == "ethereum":
    data = df3

  print("What price do you want to know?")
  print("1. Low\n2. High\n3. Open\n4.Close")
  choice = int(input())
  print("At what date and time you want to know the price?")
  print("Format : year-month-day hour(24h format, 2 digits)")
  date = pd.to_datetime(input() + ":00:00")
  specific_row = data[data['time'] == date]
  #If needed we can only return the values instead of printing
  if choice == 1:
    print("Price: " + str(specific_row['low'].iloc[0]))
  elif choice == 2:
    print("Price: " + str(specific_row['high'].iloc[0]))
  elif choice == 3:
    print("Price: " + str(specific_row['open'].iloc[0]))
  elif choice == 4:
    print("Price: " + str(specific_row['close'].iloc[0]))




In [ ]:
get_price("bitcoin")

What price do you want to know?
1. Low
2. High
3. Open
4.Close
2
At what date and time you want to know the price?
Format : year-month-day hour(24h format, 2 digits)
2
Price: 69541.89


# Bitcoin update

In [ ]:
df1

,time,low,high,open,close,volume,Bitcoin,Blockchain,buy bitcoin,crypto,cryptocurrency,unix_timestamp
0,2015-07-20 21:00:00,277.92,278.00,277.98,278.00,59.260456,3,0,0,0,0,1437426000
1,2015-07-20 22:00:00,277.37,278.50,278.00,278.14,346.995411,3,0,0,0,0,1437429600
2,2015-07-20 23:00:00,278.15,280.00,278.31,280.00,376.627552,3,0,0,0,0,1437433200
3,2015-07-21 00:00:00,279.38,281.00,279.96,280.89,288.177976,3,0,0,0,0,1437436800
4,2015-07-21 01:00:00,280.35,281.15,280.89,280.36,380.933023,3,0,0,0,0,1437440400
...,...,...,...,...,...,...,...,...,...,...,...,...
76115,2024-03-28 06:00:00,69609.86,70383.64,69688.89,70331.13,430.169089,69,4,1,29,2,1711605600
76116,2024-03-28 07:00:00,70165.93,70449.14,70342.43,70385.63,235.630213,70,4,1,27,2,1711609200
76117,2024-03-28 08:00:00,70277.17,70838.34,70386.48,70556.81,613.919833,70,4,1,30,2,1711612800
76118,2024-03-28 09:00:00,70496.78,70905.33,70556.58,70544.27,420.968324,77,4,1,34,3,1711616400


# Litecoin update

In [ ]:
df2

,time,low,high,open,close,volume,Blockchain,buy litecoin,crypto,cryptocurrency,litecoin,unix_timestamp
0,2016-08-17 04:00:00,3.70,3.70,3.70,3.70,10.000000,2,0,1,0,0,1471406400
1,2016-08-17 20:00:00,3.62,3.62,3.62,3.62,0.317691,2,0,1,0,0,1471464000
2,2016-08-18 22:00:00,3.62,3.70,3.62,3.70,1.000000,2,0,1,0,0,1471557600
3,2016-08-19 04:00:00,3.70,3.70,3.70,3.70,2.694619,2,0,1,0,0,1471579200
4,2016-08-19 20:00:00,3.70,4.00,3.70,4.00,198.838259,2,0,1,0,0,1471636800
...,...,...,...,...,...,...,...,...,...,...,...,...
65625,2024-03-28 06:00:00,95.14,96.13,95.82,96.11,12793.977343,11,0,79,6,2,1711605600
65626,2024-03-28 07:00:00,95.50,96.54,96.10,96.35,22759.855183,12,0,77,6,2,1711609200
65627,2024-03-28 08:00:00,95.47,96.55,96.35,96.11,16126.132832,11,0,83,7,2,1711612800
65628,2024-03-28 09:00:00,95.36,96.35,96.13,95.56,10611.413075,12,0,95,8,2,1711616400


# Ethereum

In [ ]:
df3

,time,low,high,open,close,volume,Blockchain,buy ethereum,crypto,cryptocurrency,ethereum,unix_timestamp
0,2016-05-18 00:00:00,12.50,14.93,12.50,13.00,8.002848,2,0,1,0,2,1463529600
1,2016-05-18 01:00:00,14.00,14.10,14.00,14.10,15.462483,2,0,1,0,2,1463533200
2,2016-05-18 02:00:00,14.00,14.00,14.00,14.00,0.052000,2,0,1,0,2,1463536800
3,2016-05-18 17:00:00,13.60,13.60,13.60,13.60,0.366730,2,0,1,0,2,1463590800
4,2016-05-18 18:00:00,13.05,13.51,13.27,13.51,166.910767,2,0,1,0,2,1463594400
...,...,...,...,...,...,...,...,...,...,...,...,...
68706,2024-03-28 06:00:00,3525.30,3574.73,3532.06,3573.90,3327.620524,11,0,79,6,24,1711605600
68707,2024-03-28 07:00:00,3561.88,3578.56,3573.58,3572.97,2857.266490,12,0,77,6,26,1711609200
68708,2024-03-28 08:00:00,3563.82,3591.64,3572.98,3574.60,2306.722459,11,0,83,7,27,1711612800
68709,2024-03-28 09:00:00,3569.81,3593.30,3574.29,3582.54,1134.982472,12,0,95,8,27,1711616400


# Saving results

In [ ]:
df1.to_csv('Bitcoin.csv', index=False)
df2.to_csv('Litecoin.csv', index=False)
df3.to_csv('Ethereum.csv', index=False)